In [1]:
# ライブラリのインポート
from textblob import TextBlob
import tweepy
import pandas as pd
import nltk
import re
import string
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from googletrans import Translator

In [2]:
consumerKey = "hCG4WejOO7GhB7b3CzF90kVNb"
consumerSecret = "0JFvkMOS14gaWcL56DOnuTUD7wc1GdqnGxuXXqhOeboPwGdXYm"
accessToken = "1500625576834396161-wjWFXEwvKVtW8lAz20Uy7QjQRRbTEJ"
accessTokenSecret = "BPUYX2rtZLAdiCvqJXnEOyNTTlTgWyEbmudy17iYJf6sC"
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

In [3]:
def percentage(part,whole):
 return 100 * float(part)/float(whole)

In [4]:
keyword = input("Please enter keyword or hashtag to search: ")
noOfTweet = int(input ("Please enter how many tweets to analyze: "))

Please enter keyword or hashtag to search:  星川サラ
Please enter how many tweets to analyze:  100


In [6]:
keyword += "-filter:retweets"

In [7]:
tweets = tweepy.Cursor(api.search_tweets, q=keyword).items(noOfTweet)

In [8]:
tweet_list = []

In [9]:
# tweets_c = []
tweets_c = [ele.text for ele in tweets if len(ele.text)>50]

In [10]:
translator = Translator()
tweet_list_original = []
for tweet in tweets_c :
    if detect(tweet)=="en" :
        tweet_list_original.append(tweet)
    else:
        tweet_list_original.append(translator.translate(tweet, dest='en').text)

In [11]:
tweet_list = pd.DataFrame(tweet_list_original)

In [22]:
tweet_list = tweet_list.drop_duplicates(subset=0)
tweet_list

,0,text,polarity,subjectivity,sentiment,neg,neu,pos,compound
0,RT @ sara_hoshikawa: 🌟 🌟 🌟 🌟 🌟 🌟 🌟\n\nHeat has...,rt _hoshikawa: 🌟 🌟 🌟 🌟 🌟 🌟 🌟\n\nheat has 39 d...,0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000
4,[Off-collaboration / summary] Senna Sanae / Su...,[off-collaboration / summary] senna sanae / su...,0.500000,0.500000,neutral,0.000,1.000,0.000,0.0000
6,Apex dating delicious with each other's fan wi...,apex dating delicious with each other's fan wi...,0.250000,0.583333,strong_positive,0.000,0.600,0.400,0.7184
9,The Bipa is in Kusosubo Hoshikawa Sarah [Nijia...,the bipa is in kusosubo hoshikawa sarah [nijia...,0.600000,0.800000,strong_positive,0.000,0.751,0.249,0.6486
15,Rt @ wotoha_staff: 202- 01/17\nA sunny JK song...,rt _staff: 202- 01/17\na sunny jk song / feat...,0.000000,0.000000,positive,0.000,0.761,0.239,0.5719
34,Advance Summon Ashikawa Sarah for sacrifice of...,advance summon ashikawa sarah for sacrifice of...,0.700000,0.600000,positive,0.000,0.789,0.211,0.5399
46,"Verifying V.\nTsukino Mikawa, Kenzo, Kagawa, H...","verifying v.\ntsukino mikawa, kenzo, kagawa, h...",0.500000,1.000000,negative,0.060,0.887,0.052,-0.0516
47,"RT @ Nijisanji_App: [Prismatic Colors ""Benefit...","rt _app: [prismatic colors ""benefits cd recor...",-0.504464,0.571429,strong_positive,0.059,0.743,0.198,0.6696
48,When I sent the GIF of Hoshikawa Sarah to Goog...,when i sent the gif of hoshikawa sarah to goog...,0.377778,0.188889,strong_positive,0.000,0.723,0.277,0.7579
57,Heat has 39 degrees...[Hoshikawa Sarah / Nijia...,heat has 39 degrees...[hoshikawa sarah / nijia...,0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000


In [23]:
tweet_list["text"] = tweet_list[0]

In [24]:
remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
rt = lambda x: re.sub("(@ [A-Za-z0–9]+)|(\w+:\/\/\S+)"," ",x)
tweet_list["text"] = tweet_list.text.map(remove_rt).map(rt)
tweet_list["text"] = tweet_list.text.str.lower()

In [25]:
tweet_list[['polarity', 'subjectivity']] = tweet_list['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tweet_list['text'].iteritems():
 score = SentimentIntensityAnalyzer().polarity_scores(row)
 neg = score['neg']
 neu = score['neu']
 pos = score['pos']
 comp = score['compound']

 if neg > pos:
  if score["compound"] < -0.6 : tweet_list.loc[index, "sentiment"] = "strong_negative"
  else : tweet_list.loc[index, 'sentiment'] = "negative"
 elif pos > neg:
  if score["compound"] > 0.6 : tweet_list.loc[index, 'sentiment'] = "strong_positive"
  else : tweet_list.loc[index, "sentiment"] = "positive"
 else:
  tweet_list.loc[index, 'sentiment'] = "neutral"
 tweet_list.loc[index, 'neg'] = neg
 tweet_list.loc[index, 'neu'] = neu
 tweet_list.loc[index, 'pos'] = pos
 tweet_list.loc[index, 'compound'] = comp
tweet_list.tail(10)

,0,text,polarity,subjectivity,sentiment,neg,neu,pos,compound
15,Rt @ wotoha_staff: 202- 01/17\nA sunny JK song...,rt _staff: 202- 01/17\na sunny jk song / feat...,0.000000,0.000000,positive,0.000,0.761,0.239,0.5719
34,Advance Summon Ashikawa Sarah for sacrifice of...,advance summon ashikawa sarah for sacrifice of...,0.700000,0.600000,positive,0.000,0.789,0.211,0.5399
46,"Verifying V.\nTsukino Mikawa, Kenzo, Kagawa, H...","verifying v.\ntsukino mikawa, kenzo, kagawa, h...",0.500000,1.000000,negative,0.060,0.887,0.052,-0.0516
47,"RT @ Nijisanji_App: [Prismatic Colors ""Benefit...","rt _app: [prismatic colors ""benefits cd recor...",-0.504464,0.571429,strong_positive,0.059,0.743,0.198,0.6696
48,When I sent the GIF of Hoshikawa Sarah to Goog...,when i sent the gif of hoshikawa sarah to goog...,0.377778,0.188889,strong_positive,0.000,0.723,0.277,0.7579
57,Heat has 39 degrees...[Hoshikawa Sarah / Nijia...,heat has 39 degrees...[hoshikawa sarah / nijia...,0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000
70,Heat has 39 degrees...[Hoshikawa Sarah / Nijia...,heat has 39 degrees...[hoshikawa sarah / nijia...,0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000
72,"""Each other is the best air with each other's ...","""each other is the best air with each other's ...",0.250000,0.350000,strong_positive,0.000,0.775,0.225,0.7506
74,[Fan videos] Men's Chinese food [VTUBER Chines...,[fan videos] men's chinese food [vtuber chines...,0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000
81,[Fan videos] Hoshikawa Sara [Nijians / cutout]...,[fan videos] hoshikawa sara [nijians / cutout]...,0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000


In [26]:
def count_values_in_column(data,feature):
 total=data.loc[:,feature].value_counts(dropna=False)
 percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
 return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

In [27]:
count_values_in_column(tweet_list, "sentiment")

,Total,Percentage
neutral,6,42.86
strong_positive,5,35.71
positive,2,14.29
negative,1,7.14
